# Image Conver from color to grayscale test
A quick intro to using the pre-trained model to detect and segment object of person.

In [1]:
from os import path
import sys
import random
import math

import warnings
warnings.filterwarnings('ignore')

import SimpleITK as sitk
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import sys

from keras.utils import print_summary
# Add the ptdraft folder path to the sys.path list
sys.path.append('../')

import capsnet as modellib
from data_helper import *
from test import *
from load_2D_data import *
from PIL import Image

%matplotlib inline 

# Root directory of the project
ROOT_DIR = path.dirname("../")
DATA_DIR = path.join(ROOT_DIR, "data")

# Directory to save logs and trained model
MODEL_DIR = path.join(DATA_DIR, "saved_models/segcapsr3/m1.hdf5") # LUNA16

# Local path to trained weights file
COCO_MODEL_PATH = path.join(DATA_DIR, "saved_models/segcapsr3/mc3.hdf5") # MSCOCO17

# Directory of images to run detection on
IMAGE_DIR = path.join(DATA_DIR, "imgs")
MASKS_DIR = path.join(DATA_DIR, "masks")

Using TensorFlow backend.


## Read image



In [4]:
from datetime import datetime

img_name = ['train15.png']
output_array = None


itk_img = sitk.ReadImage(join(IMAGE_DIR, img_name[0]))
img = sitk.GetArrayFromImage(itk_img)

# The source image should be 512X512 resolution.
img = image_resize2square(img, IMAGE_SIZE)
# TODO computing by color image
#         img = img[:,:,:3] # Only get RGB channels. Remove alpha channel.
# Translate the image to grayscale
img2 = Image.fromarray(img).convert('L')
# plt.imshow(img2)
img2.show()
img = np.array(Image.fromarray(img).convert('L'))  

print(img.shape)
img = img.reshape([img.shape[0], img.shape[1], 1])
print(img.shape)
Image.fromarray(img[:,:,0]).show()


(512, 512)
(512, 512, 1)


# Mask image testing



In [7]:
itk_mask = sitk.ReadImage(join(MASKS_DIR, img_name[0]))
mask = sitk.GetArrayFromImage(itk_mask)
mask = image_resize2square(mask, IMAGE_SIZE)

# print(mask)
mask = change_background_color(mask, COCO_BACKGROUND, MASK_BACKGROUND) 
print(mask)
img2 = Image.fromarray(mask)
img2.show()

# mask = np.array(Image.fromarray(mask).convert('L'))  
#   mask = mask[:,:,:3] # Only get RGB channels. Remove alpha channel.      
mask = mask[:,:,:1]

mask[mask >= 1] = 1 # The mask. ie. class of Person
mask[mask != 1] = 0 # Non Person / Background

mask = mask.astype(np.uint8)


[[[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 ...

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]
